In [1]:
'''
600-750 for height 0.15 +/- 0.025
H0
'''

'\n600-750 for height 0.15 +/- 0.025\nH0\n'

### Imports modules and vector fields

In [2]:
from jupyterthemes import jtplot
jtplot.style(theme = "monokai", context='notebook', ticks=True, grid=False)
#Makes plotting compatible with the Jupyter-notebook dark theme 'monokai'

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from importlib import reload
from scipy import interpolate
import PIVutils
import PODutils
import grafteaux as G
import automateG as AG
from scipy.ndimage.measurements import find_objects, label, center_of_mass
import prom2d as PP
import time

G = reload(G)
X, Y, U, V,Swirl, Cond,Prof, SwirlFiltPro, SwirlFiltRet, SwirlFilt = G.init_data()

Umean = Prof['U']

xog = X[0,:]
yog = Y[:,0]

del_x = (xog[0]-xog[1])
ar_th  = del_x*del_x*np.pi*25


current path : c:\program files\python37\python.exe
['#refs#', 'AutoCorr', 'Cond', 'Corr', 'CorrT', 'Err', 'PercentMissing', 'Prof', 'Quad', 'Swirl', 'U', 'V', 'W', 'X', 'Y', 'source']


C:\Users\Kommalapati sahil\AppData\Roaming\Python\Python37\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


0.2439601862683396


### Evaluate the T2 field of the entire frame(s) and find the containing blob radius and prominences


In [4]:
umean2  = np.array([Umean[:,0] for i in range(U.shape[1])]).T

In [5]:
frame1, frame2  = 600, 750


In [7]:
#u2d = np.nanmean(U, axis = 2) #local - pixel mean velocity

centers = {}
proms = {}
areas = {}

t2s = {} #contains the T2 field at that the frame

#evaluating from frame1 to frame2.
#frame1, frame2  = 250, 400

for frame in range(frame1, frame2):
    s1 = time.time()

    t2f = G.T2_big_mat(X, Y, U[:,:,frame]-umean2, V[:,:,frame], 2)
    
    #t2f cleaning 
    t21 = t2f.copy()
    t21[t2f<0] = 0 #only prograde 
    t21[t2f<2/np.pi/1.5] = 0 #adding some preliminary threshold 0.66%2/pi
    
    #from every frame collect centers and proms
    
    t2s[frame] = t21
    
    t2_obs= find_objects(label(t21)[0]) #slice objects
    t2_coms_ = center_of_mass(t21, label(t21)[0], [i+1 for i in range(len(t2_obs))]) #center of masses
    t2_coms = np.array(t2_coms_).astype(int)
    print('Beginning..')
    peaks,idmap,promap,parentmap = PP.getProminence(t21,0.01,lats=yog,\
                                                    lons=xog,\
                                                    min_area=ar_th,\
                                                    include_edge=True,\
                                                    verbose = False)
    
    print(f'Frame - {frame} done.' )
    
    cent = []
    prom = []
    aa = []
    for i in peaks.keys():
        
        prom_ = peaks[i]['prominence']
        cc = PP.getind(peaks[i]['center'],xog, yog)
        aa_ = peaks[i]['area']
        cc1 = (cc[1], cc[0])
        
        cent.append(cc1)
        prom.append(prom_)
        aa.append(aa_)
    
    centers[frame] = cent
    proms[frame] = prom
    areas[frame] = aa
    
    s2 = time.time()
    print(f'took {round(s2 - s1,2)} secs.')
    
    #if frame+1 >= fr_th:     break

#we got centers and proms

Beginning..
Frame - 600 done.
took 480.61 secs.


C:\Users\Kommalapati sahil\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less
C:\Users\Kommalapati sahil\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


Beginning..
Frame - 601 done.
took 1232.27 secs.
Beginning..
Frame - 602 done.
took 556.06 secs.
Beginning..
Frame - 603 done.
took 569.49 secs.
Beginning..
Frame - 604 done.
took 1642.36 secs.
Beginning..
Frame - 605 done.
took 426.75 secs.
Beginning..
Frame - 606 done.
took 457.27 secs.
Beginning..
Frame - 607 done.
took 419.89 secs.
Beginning..
Frame - 608 done.
took 1160.66 secs.
Beginning..
Frame - 609 done.
took 536.94 secs.
Beginning..
Frame - 610 done.
took 1162.45 secs.
Beginning..
Frame - 611 done.
took 585.18 secs.
Beginning..
Frame - 612 done.
took 450.63 secs.
Beginning..
Frame - 613 done.
took 465.23 secs.
Beginning..
Frame - 614 done.
took 452.21 secs.
Beginning..
Frame - 615 done.
took 830.51 secs.
Beginning..
Frame - 616 done.
took 437.61 secs.
Beginning..
Frame - 617 done.
took 423.85 secs.
Beginning..
Frame - 618 done.
took 576.54 secs.
Beginning..
Frame - 619 done.
took 516.88 secs.
Beginning..
Frame - 620 done.
took 1140.25 secs.
Beginning..
Frame - 621 done.
took 

In [6]:
#props = [t2s, centers, proms, areas, bbdims]
props = np.load('props600_750.npy', allow_pickle = True)
[t2s, centers, proms, areas, bbdims] = props 

In [7]:
#props600_750 = [t2s, centers, proms, areas, bbdims]
#np.save('props600_750.npy', props600_750, allow_pickle = True)

In [8]:
#join props0_100 with props100_150
#then implement h0 for props 0_150
#then implement h1 for props 0_150
#then implement h2 for props 0_150


### Functions to get the half prominence regions from the prominence values

In [9]:


def getdims(G):
    '''
    has 1s everywhre
    and 2 at center
    returns ( by-top, by-down, bx-left, bx-right)
    '''
    
    c = np.where(G==2)
    cx, cy  = c[0][0], c[1][0]
    return (cx, len(G)-cx-1, cy, len(G[0])-cy-1)
    
def find_blobs(G, cent, prom, ratio = 0.5, verbose = True):
    '''
    G- the big field
    cent- blob center whose bounds we want
    
    Very naive implementation - can be made significantly faster
    '''
    
    #modify G with prom and ratio
    pr_th = G[cent]-prom*ratio
    G_ = G.copy()
    G_[G<=pr_th] = 0 #only regions with half prom available
    
    G_obs = find_objects(label(G_)[0])
    #print(len(G_obs))
    for i in G_obs:
        #print(i)
        test  = np.zeros(G.shape)
        test[i] = 1
        if test[cent]:
            #this blob contains the cnt
            if verbose:
                plt.figure()
                plt.imshow(test)
                plt.colorbar()
            test[cent] = 2
            dims = getdims(test[i])
            return dims
    return -1

    #indicates that the search was failure 
    #unlikelyy - probably some rounding error somewhere


### Get the extent of the half prominence region from the center

In [10]:
#centers.keys()

In [11]:
##bbdims array - tales centers and proms and 
#gets the half prom (or some custom ratio) blob regions.

#can set verbose in find_blobs = False to not visualize the blobs

bbdims = {}
for i in centers.keys():
    t21 = t2s[i]
        
    #for each frame
    bbcurr = []
    #print('sdfg')
    for cf in range(len(centers[i])):
        bb_ = find_blobs(t21, centers[i][cf], proms[i][cf], ratio = 1, verbose  = False)
        bbcurr.append(bb_)
        #print(bb_)
        #break
    bbdims[i] = bbcurr
    

C:\Users\Kommalapati sahil\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in less_equal


### Evaluating MCMC on the localized fields

In [12]:
#we have global cents, global blob dims exact, frame 
#mcmc part 

ct = 0
ct_lim = 10

props_dir = {}

mc_soldir = {}
min_soldir = {}

mc_resdir = {}
min_resdir = {}

samp_dir = {}

height_th = [U.shape[0]*0.125, U.shape[0]*0.175]

In [13]:
#newBBdims, newCents
#calculate the min sol and then compare with mc sol.

from scipy import optimize

y = yog.copy()
x = xog.copy()

SR = 5
bounds = [(0.01*2*np.pi*(x[1]-x[0]), None), (x[1]-x[0],np.max(x)),\
          (-1*SR*(x[1]-x[0]),SR*(x[1]-x[0])),(-1*SR*(x[1]-x[0]),SR*(x[1]-x[0])),\
          (-0.15, 0.15)]


Circ = 0.08 #circulation strength
r = 0.03    #radius
xc = 0.02  #center.x 
yc = 0.02   #center.y
Conv = 0 #convective velocity

target = [Circ,r,xc,yc,Conv]

In [14]:
AG = reload(AG)

In [15]:
def calError(props,cen, f, U, V, Cond, k=1.5):
    #return the difference using r
    s, r, x, y, v = props
    yo, xo = cen
    
    dx = 0.010118516377207676
    #print(r)
    x = int(round(x/dx))
    y = int(round(y/dx))
    
    #print(x,y)
    r*= k
    r = int(round(r/dx))
    #print(r)
    
    x+= xo
    y+= yo
    #print(x, y , r)
    if y-r>=0 and y+r < len(U) and x-r>=0 and x+r+1 < len(U[0]):

        U1 = U[y-r:y+r+1, x-r:x+r+1,f]
        V = V[y-r:y+r+1, x-r:x+r+1,f]
        Un = (U1-Umean[y])/Cond['Uinf']
        
        x1, y1  = G.get_xy_rect(Un.shape[0], Un.shape[1])
        #print(x1.shape, Un.shape)
        m_res = PODutils.minfuncVecField10r_rect(props, Un, V, x1, y1)
        return m_res
    else : 
        #print(y-r>=0 , y+r < len(U) , x-r>=0 , x+r+1 < len(U[0]))
        return 0  
    

In [16]:
# m_res = calError(match.x, center, frame, U, V,Cond, k = 1.5 )

In [ ]:

ct = 0
ct_lim = 1
bc = 0
f3_count = 0
AG = reload(AG)

ii = 0
for i in centers.keys():
    #for each frame 
    print(f'frame-{ii}'); ii+=1
    for  j in range(len(centers[i])):
        
        if bbdims[i][j] == -1 : print('bbdims fail.'); continue 
        
        print(f'working on vrtex - {ct}.')
        b1, b2, b3, b4 = bbdims[i][j]
        cy, cx = centers[i][j]
        
        if cy >= height_th[0] and cy <= height_th[1]: 
            
                
            if not (b1+b2+1) %2 : b2 +=1
            if not (b3+b4+1) %2 : b4 +=1

            uu = U[cy-b1:cy+b2+1, cx-b3:cx+b4+1,i]
            vv = V[cy-b1:cy+b2+1, cx-b3:cx+b4+1,i]
            ss = Swirl[cy-b1:cy+b2+1, cx-b3:cx+b4+1,i]

            uun = (uu-Umean[cy])/Cond['Uinf']

            x1, y1  = G.get_xy_rect(uu.shape[0], uu.shape[1])
            
            samp_test = G.doMCMC_V4(uun, vv, x1, y1)
            
            bbs = bbdims[i][j]
            center = centers[i][j]
            frame = i 
            
            p1 = {}
            p1['bbdims'] = np.array(bbs)
            print(bbs)
            p1['cent'] = center
            p1['frame'] = frame
            props_dir[ct] = p1 
            
            f3 = AG.F3check(samp_test,5000,95)
            
            init = target.copy()

            match = optimize.minimize(PODutils.minfuncVecField10r_rect, x0=init, args=(uun,vv,x1,y1) ,bounds=bounds,\
                             method = 'TNC',options={'maxiter':100000,'disp': True}) #'gtol': 1e-6,

            min_soldir[ct] = match.x
            #m_res = PODutils.minfuncVecField10r_rect(match.x, uun, vv, x1, y1)
            m_res = calError(match.x,center, frame, U, V,Cond, k = 1.5 )
            min_resdir[ct] = m_res
            
            if not f3: 
                #store sampler, mc sol, mc res, min sol, min res
                bc +=1 
                #samp_dir[ct] = samp_test
                mc_soldir[ct] = AG.cornerSol(samp_test, 5000)
                mc_res = calError(mc_soldir[ct], center, frame, U, V,Cond, k = 1.5 )
                #mc_resdir[ct] = PODutils.minfuncVecField10r_rect(mc_soldir[ct], uun, vv, x1, y1)
                mc_resdir[ct] = mc_res
                
                
            else: 
                
                f3_count+=1 
                #do corrections et al. 
                print(f'Corrections on the vortex - {ct}.')
                props = props_dir[ct]
                solZip = AG.DcorrectionV4(Cond, U, V, Umean, props, samp_test)
                if solZip !=  -1:
                    #samp_dir[ct] = solZip[0]
                    props_dir[ct]['bbdims'] = solZip[1]
                    mc_resdir[ct] = solZip[2]
                else: 
                    #samp_dir[ct] = samp_test 
                    mc_resdir[ct] = -1 
            print(f'vortex - {ct} done.')
            del samp_test 
            ct+=1
            #if ct>= ct_lim: break
    #else: continue 
    #break #comes here when break in loop executed
        

frame-0
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
working on vrtex - 0.
(3, 4, 7, 3)
Corrections on the vortex - 0.
working on rad: (4, 6, 8, 4)  rth- 14


In [ ]:
AG = reload(AG)

In [ ]:
lkeys = [i for i in mc_soldir.keys() if mc_resdir[i] != -1]

i1 = lkeys[0]
mc_sols = []
for i1 in lkeys:
    stemp = mc_soldir[i1]
    mc_sols.append(stemp)

In [ ]:
props_save = [props_dir[i] for i in lkeys]
#mc_save = np.array([AG.cornerSol(samp_dir[i], 5000) for i in lkeys ])
mn_save = [min_soldir[i] for i in lkeys]

In [ ]:
Nsave = np.array([props_save, mc_sols, mn_save])
np.save(f'SML_H0_F{frame1}-{frame2}.npy', Nsave, allow_pickle = True)